In [ ]:
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.gridspec import GridSpec

from fiat_toolbox.well_being import Household


Let's assume we have a household that has a house with a building structure value of '**k_str**' that is damaged by a flood leading to a repair costs '**v**', representing the ratio of the structure value (i.e., v=0.7 describes repair costs that are 70% of the building structure value). Let's assume that the annual consumption of the household '**c0**' is equal to the annual income of the household. And that the average consumption of the area of interest is '**c_avg**' (represented by the average income)

In [ ]:
v = 0.7 # fraction of damage to the building
k_str = 30000 # building value
c0 = 48000 # annual income
c_avg = 48000 # average annual income

We can define some economic indicators to assess specific type of losses.

In [ ]:
pi = 0.15 # productivity of capital index
eta = 1.5 # elasticity of the marginal utility
rho = 0.06 # discount rate
t_max = 10 # recovery time in years
dt = 1/52 # time step in years
currency = "$"

Then we can use the Household class to define a household with these characteristics.

In [ ]:
# Create a WellBeing object and optimize the lambda value
household = Household(v=v, k_str=k_str, c0=c0, c_avg=c_avg, pi=pi, eta=eta, rho=rho, dt=dt, t_max=t_max, currency=currency)
household

We could have provided already a reconstruction rate with the argument "l". Since this is normally not known in advance, we can actually calculate the recovery rate that would minimize the utility loss of the household using the "opt_lambda" method. We can visualize the optimization with the "plot_opt_lambda" method.

In [ ]:
household.opt_lambda(method="trapezoid")
household.plot_opt_lambda(x_type="time");

There are two methods that can be used for calculating the utility loss of the household. 

- The "trapezoid" method, which calculates the integral based on the trapezoid rule, using the numpy package (faster but accurate only for small enough time steps)
- The "quad" method, which calculates the integral based on the adaptive quadrature algorithm, using the scipy package (slower but not dependent on time step)

As you can see in the graph below, the trapezoid method is not accurate enough for large time steps. The quad method is more accurate and should be used for larger time steps. The lack of accuracy of the trapezoid method is high for the really high recovery rates were essentially the recovery time is very short.

In [ ]:
# Measure execution time for trapezoid method
start_time = time.time()
household.opt_lambda(method="trapezoid")
end_time = time.time()
trapezoid_time = end_time - start_time

l = household.l_opt["lambda"]
z1 = household.l_opt["Utility Loss"]

# Measure execution time for quad method
start_time = time.time()
household.opt_lambda(method="quad")
end_time = time.time()
quad_time = end_time - start_time

z2 = household.l_opt["Utility Loss"]

# Plot the results
plt.plot(l, z1, label=f'Trapezoid Method ({trapezoid_time:.2f}s)')
plt.plot(l, z2, label=f'Quad Method ({quad_time:.2f}s)')
plt.xlabel('Lambda')
plt.ylabel('Utility Loss')
plt.legend()
plt.show()

We can run the two methods multiple times for different combinations of time-steps to compare the accuracy of the trapezoid method versus the computation time.

In [ ]:
# Define the bins for lambda and dt
lambda_bins = np.linspace(0.29, 2.3, 9)
dts = np.linspace(0.01, 1, 10)

# Create a DataFrame to store the average relative differences
bin_labels = [f"{lambda_bins[i]:.2f}-{lambda_bins[i+1]:.2f}" for i in range(len(lambda_bins) - 1)]
heatmap_data = pd.DataFrame(index=dts, columns=bin_labels)
computation_times = []
# Calculate the average relative difference for each bin
for k, dt in enumerate(dts):
    household = Household(v=v, k_str=k_str, c0=c0, c_avg=c_avg, t_max=10, dt=dt)
    
    start_time = time.time()
    household.opt_lambda(method="trapezoid")
    end_time = time.time()
    t1 = end_time - start_time
    
    l = household.l_opt["lambda"]
    z1 = household.l_opt["Utility Loss"]
    
    start_time = time.time()
    household.opt_lambda(method="quad")
    end_time = time.time()
    t2 = end_time - start_time

    z2 = household.l_opt["Utility Loss"]
    
    rel_diff = np.abs(z1 - z2) / np.abs(z2)
    for i in range(len(lambda_bins) - 1):
        mask = (l >= lambda_bins[i]) & (l < lambda_bins[i + 1])
        rel_diff_mean = rel_diff[mask].mean() if mask.any() else np.nan
        heatmap_data.iloc[k, i] = rel_diff_mean*100
    
    computation_times.append(t1/t2)  # Store computation gain

# Convert the data to float for plotting
heatmap_data = heatmap_data.astype(float)

In [ ]:

# Create a figure with two subplots side by side
fig = plt.figure(figsize=(14, 8))
gs = GridSpec(1, 2, width_ratios=[3, 1], wspace=0.3)

# Plot the heatmap on the left
ax1 = fig.add_subplot(gs[0, 0])
sns.heatmap(heatmap_data, xticklabels=bin_labels, yticklabels=dts, cbar_kws={'label': 'Mean Absolute Difference (%)'}, ax=ax1, vmin=0, vmax=100, cmap='coolwarm')
ax1.set_xlabel("recovery rate λ")
ax1.set_ylabel("time step (years)")
ax1.set_title("Heatmap of Average Relative Difference")
ax1.set_xticks(ax1.get_xticks())
ax1.set_xticklabels(ax1.get_xticklabels(), rotation=45)  # Rotate x-axis tick labels for better visibility

# Plot the line graph on the right
ax2 = fig.add_subplot(gs[0, 1])
ax2.plot(computation_times, dts, marker='o', label='Comp. time ratio')
ax2.set_xlabel("trapezoid/quad comp.time ratio (-)")
ax2.set_ylabel("time step (years)")
ax2.set_title("Computation Ratio vs Time Step")
ax2.set_yticks(dts)  # Align y-axis ticks with the heatmap
ax2.grid(True)
ax2.legend()
ax2.invert_yaxis()  # Reverse the y-axis

plt.show()

So, this shows that even with small dt the trapezoid is accurate while it is still a lot faster when compared to the quad method.

So let's run an analysis with a daily time step and the trapezoid method for a single household. We can then show the total losses over the recovery time, with the get_losses method.

In [ ]:
household = Household(v=v, k_str=k_str, c0=c0, c_avg=c_avg, t_max=10, dt=1/365)
household.opt_lambda(method="trapezoid")
household.get_losses("trapezoid")

We can plot the consumption over time after the disaster with the "plot_consumption" method.

In [ ]:
household.plot_consumption()